In [35]:
import pandas as pd
from urllib.parse import urlparse
from tld import get_tld
import os.path
from ssl_check import STL

fil2 = STL(path="url_t.csv")
# print(fil2)

url = pd.read_csv("url_t.csv")
# length of URL
url['url_length'] = url['url'].apply(lambda i: len(str(i)))
#length of hostname
url['hostname_length'] = url['url'].apply(lambda i: len(urlparse(i).netloc))
#Path Length
url['path_length'] = url['url'].apply(lambda i: len(urlparse(i).path))

#First Directory Length
def fd_length(url):
    urlpath= urlparse(url).path
    try:
        return len(urlpath.split('/')[1])
    except:
        return 0

# print(fd_length(url = 'https://www.bilibili.com/?spm_id_from=333.788.0.0'))

url['fd_length'] = url['url'].apply(lambda i: fd_length(i))

#Length of Top Level Domain
url['tld'] = url['url'].apply(lambda i: get_tld(i,fail_silently=True))
def tld_length(tld):
    try:
        return len(tld)
    except:
        return -1

url['tld_length'] = url['tld'].apply(lambda i: tld_length(i))

url['count-'] = url['url'].apply(lambda i: i.count('-'))
url['count@'] = url['url'].apply(lambda i: i.count('@'))
url['count?'] = url['url'].apply(lambda i: i.count('?'))
url['count%'] = url['url'].apply(lambda i: i.count('%'))
url['count.'] = url['url'].apply(lambda i: i.count('.'))
url['count='] = url['url'].apply(lambda i: i.count('='))
url['count-http'] = url['url'].apply(lambda i : i.count('http'))
url['count-https'] = url['url'].apply(lambda i : i.count('https'))
url['count-www'] = url['url'].apply(lambda i: i.count('www'))

def digit_count(url):
    digits = 0
    for i in url:
        if i.isnumeric():
            digits = digits + 1
    return digits
url['count-digits']= url['url'].apply(lambda i: digit_count(i))

def letter_count(url):
    letters = 0
    for i in url:
        if i.isalpha():
            letters = letters + 1
    return letters
url['count-letters']= url['url'].apply(lambda i: letter_count(i))

def no_of_dir(url):
    urldir = urlparse(url).path
    return urldir.count('/')
url['count_dir'] = url['url'].apply(lambda i: no_of_dir(i))

# url.head()
import re
#Use of IP or not in domain
def having_ip_address(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)' # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', url)  # Ipv6
    if match:
        # print match.group()
        return -1
    else:
        # print 'No matching pattern found'
        return 1
url['use_of_ip'] = url['url'].apply(lambda i: having_ip_address(i))

def shortening_service(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      url)
    if match:
        return -1
    else:
        return 1
url['short_url'] = url['url'].apply(lambda i: shortening_service(i))
url.shape
x = url[['hostname_length',
       'path_length', 'fd_length', 'tld_length', 'count-', 'count@', 'count?',
       'count%', 'count.', 'count=', 'count-http','count-https', 'count-www', 'count-digits',
       'count-letters', 'count_dir', 'use_of_ip']]
x.shape
# print(x)
f = 1
uurl = "github" #!prepared 
uul = url['url'].to_string()
# print(type(uul))
if uurl in uul:
    f = 0
# define the modle of randomforest
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth=100)

import joblib
rfc = joblib.load('rfc_model.joblib')
# accuracy_score(y_test, rfc_predictions)
rfc_predictions = rfc.predict(x)
#! clear remain infomation
# os.system("clear")
# print(type(rfc_predictions.tolist()[0]))

A = rfc_predictions.tolist()[0]

# print(A)
# if fil2 == 1:
#     print("valid url {}".format(uul))
# else:
uuul = uul.split(" ")[-1]

if fil2 == 1:
    if A == 0:
        print("valid url {}".format(uuul))
    elif f==0:
        print("valid url {}".format(uuul))
    else:
        print("mallicious url")
else:
    print("mallicious url")


{'subject': ((('countryName', 'US'),), (('stateOrProvinceName', 'California'),), (('localityName', 'San Francisco'),), (('organizationName', 'GitHub, Inc.'),), (('commonName', 'github.com'),)), 'issuer': ((('countryName', 'US'),), (('organizationName', 'DigiCert Inc'),), (('commonName', 'DigiCert TLS Hybrid ECC SHA384 2020 CA1'),)), 'version': 3, 'serialNumber': '0CD0A8BEC632CFE645ECA0A9B084FB1C', 'notBefore': 'Feb 14 00:00:00 2023 GMT', 'notAfter': 'Mar 14 23:59:59 2024 GMT', 'subjectAltName': (('DNS', 'github.com'), ('DNS', 'www.github.com')), 'OCSP': ('http://ocsp.digicert.com',), 'caIssuers': ('http://cacerts.digicert.com/DigiCertTLSHybridECCSHA3842020CA1-1.crt',), 'crlDistributionPoints': ('http://crl3.digicert.com/DigiCertTLSHybridECCSHA3842020CA1-1.crl', 'http://crl4.digicert.com/DigiCertTLSHybridECCSHA3842020CA1-1.crl')}
valid url https://github.com/small-cai/QR-code-scanning-...
